In [25]:
import numpy as np
import scipy as sp

# Question 13

In [15]:
H = np.array([-0.8,0.2]) # high loss prob
I = np.array([0.15,0.85])
las = np.array([[1/55, 1/125],[1/80, 1/100]])
t = sp.symbols('t', positive=True)
cost_dist = 0
for i in range(2):
    for j in range(2):
        la = las[i][j]
        cost_dist+=H[i]*I[j]*la*sp.exp(-la*t)

print("cost_dist", cost_dist)
mean = sp.integrate(t*cost_dist, (t, 0, sp.oo))
var = sp.integrate(t**2*cost_dist, (t, 0, sp.oo)) - mean**2
print("var", var)
print("mean", mean)

cost_dist -0.00218181818181818*exp(-0.0181818181818182*t) + 0.000375*exp(-0.0125*t) + 0.0017*exp(-0.01*t) - 0.00544*exp(-0.008*t)
var -23404.8400000000
mean -72.2000000000000


In [17]:
import numpy as np

a = np.array([-0.00218181818181818, 0.000375, 0.0017, -0.00544])
lam = np.array([0.0181818181818182, 0.0125, 0.01, 0.008])

area = np.sum(a/lam)          # = 1.0 here
mean = np.sum(a/lam**2) / area
var = np.sum(a*2/lam**3) / area - mean**2
mean,var

(120.33333333333337, 15839.888888888887)

In [10]:
area, mean, var

(-0.7000000000000001, -0.013852597402597401, -0.00014571423047731488)

#Q16

eq 1.31

In [21]:
Q = np.array([[-1,1,0,0,0],[0,-1,0.9,0.1,0],[0,0,-1,0,1],[0,0,0,-1/11,1/11],[0,0,0,0,0]])
S = Q[:-1,:-1]
U = np.linalg.inv(-S)
alpha = np.array([1,0,0,0])
one = np.ones((4,1))
t_mean = alpha @ U @ one
t_mean

array([4.])

In [30]:
1-alpha @ sp.linalg.expm(S*4) @ one

array([0.70271186])

In [43]:
S

array([[-1.        ,  1.        ,  0.        ,  0.        ],
       [ 0.        , -1.        ,  0.9       ,  0.1       ],
       [ 0.        ,  0.        , -1.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , -0.09090909]])

In [44]:
import numpy as np

S_big = np.kron(np.eye(12), S)

alpha_big = np.zeros((1, S_big.shape[0]))
one_big = np.ones((S_big.shape[0], 1))

1-alpha_big @ sp.linalg.expm(S_big*60) @ one_big

array([[1.]])

In [ ]:
1-alpha @ sp.linalg.expm(S*4) @ one